# 📊 **Data Mining Preliminary Project**

**Group Members:**
- Nathanael Chris Abrigo
- Dwight Kenneth Cruz
- Joshua Kyle Kessel Entrata
- Edjin Jerney Payumo


##### Install Dependencies

In [ ]:
### Install the required packages 
# Uncomment the following line to install the required packages

# !pip install pandas numpy seaborn matplotlib plotly scikit-learn nbformat


In [ ]:
%matplotlib inline

### Importing Libraries

# Data manipulation and analysis
import pandas as pd
import numpy as np

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px


In [ ]:
# Loading the 2023 FIES Dataset Volume 2
fies_23 = pd.read_csv('../data/raw/FIES PUF 2023 Volume2 Household Summary.csv')

In [ ]:
# Displaying the first 10 rows of the dataset
pd.set_option('display.max_columns', None)
fies_23.head(10)

In [ ]:
# Displaying the shape of the dataset
fies_23.shape

In [ ]:
# Displaying the columns of the dataset
fies_23.columns

In [ ]:
# Displaying the data types of the columns and the number of non-null values
fies_23.info()

In [ ]:
# Displaying the summary statistics of the dataset
fies_23.describe()

In [ ]:
### Displaying the missing values in the dataset

# Calculating the Missing Values % contribution in DF
df_null = round(100*(fies_23.isnull().sum())/len(fies_23), 2)

# Plotting the df_null
plt.figure(figsize=(16,8))
sns.barplot(x=df_null.index, y=df_null.values, alpha=0.8)
plt.title('Missing Values (Pre-Cleaning)')
plt.ylabel('Missing Values %')
plt.xlabel('Columns')
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Identifying duplicate rows using `duplicated()` method
duplicate_rows = fies_23[fies_23.duplicated()]
print(f"Number of duplicate rows: {duplicate_rows.shape[0]}")


In [ ]:
# Identifying duplicate rows by comparing total rows with unique rows
total_rows = fies_23.shape[0]
unique_rows = fies_23.drop_duplicates().shape[0]
duplicate_rows = total_rows - unique_rows

print(f"Total rows: {total_rows}")
print(f"Unique rows: {unique_rows}")
print(f"Duplicate rows: {duplicate_rows}")


### Eda


In [ ]:
cleaned_fies = fies_23.copy()

In [ ]:
def check_aggregation(
    dataframe, components, aggregated_column, negative_components=None, temp_column_name="TEMP_SUM", tolerance=0.0
):
    # Validate input columns
    all_columns = components + (negative_components if negative_components else []) + [aggregated_column]
    missing_columns = [col for col in all_columns if col not in dataframe.columns]
    if missing_columns:
        raise ValueError(f"Missing columns in the DataFrame: {missing_columns}")

    temp_df = dataframe.copy()

    # Convert relevant columns to float64 to ensure consistency
    temp_df[aggregated_column] = temp_df[aggregated_column].astype(np.float64)
    
    # Calculate the sum of positive components
    temp_df[temp_column_name] = temp_df[components].sum(axis=1)

    # Subtract negative components if provided
    if negative_components:
        temp_df[temp_column_name] -= temp_df[negative_components].sum(axis=1)

    # Use np.isclose to check if values are within the tolerance
    temp_df['MATCH'] = np.isclose(temp_df[aggregated_column], temp_df[temp_column_name], atol=tolerance)

    # Identify mismatched rows
    mismatched_rows = temp_df[~temp_df['MATCH']][[aggregated_column, temp_column_name]]

    # Output results
    if mismatched_rows.empty:
        result = f"The '{aggregated_column}' column is correctly aggregated from its components within a tolerance of ±{tolerance}."
    else:
        result = f"Number of mismatched rows: {len(mismatched_rows)}"

    return mismatched_rows, result

In [ ]:
# List of possible component columns
components = [
    'REG_SAL', 'SEASON_SAL'
]

loss_components = []
aggregate_column = 'WAGES'

# Call the function verify the aggregated column
mismatched_rows, result = check_aggregation(cleaned_fies, components, aggregate_column, negative_components=loss_components)

# Display the result
print(result)

# If there are mismatches, display them
if not mismatched_rows.empty:
    print("Mismatched rows:")
    print(mismatched_rows[[aggregate_column, 'TEMP_SUM'] + components])
else:
    Wages_components = components

Wages_components

In [ ]:
# List of possible component columns
components = [
    'NET_CFG', 'NET_LPR', 'NET_FISH', 'NET_FOR', 
    'NET_RET', 'NET_MFG', 'NET_TRANS', 'NET_NEC_A8', 
    'NET_NEC_A9', 'NET_NEC_A10', 'LOSSES'
]

loss_components = []
aggregate_column = 'EAINC'

# Call the function verify the aggregated column
mismatched_rows, result = check_aggregation(cleaned_fies, components, aggregate_column, negative_components=loss_components)

# Display the result
print(result)

# If there are mismatches, display them
if not mismatched_rows.empty:
    print("Mismatched rows:")
    print(mismatched_rows[[aggregate_column, 'TEMP_SUM'] + components])
else:
    EAINC_components = components

EAINC_components

In [ ]:
# List of possible component columns
components = [
    'WAGES', 'NETSHARE', 'CASH_ABROAD', 'CASH_DOMESTIC', 'RENTALS_REC', 
    'INTEREST', 'PENSION', 'DIVIDENDS', 'OTHER_SOURCE', 'NET_RECEIPT', 'REGFT',
    'IMPUTED_RENT', 'EAINC'
]

loss_components = []
aggregate_column = 'TOINC'

# Call the function verify the aggregated column
mismatched_rows, result = check_aggregation(cleaned_fies, components, aggregate_column, negative_components=loss_components)

# Display the result
print(result)

# If there are mismatches, display them
if not mismatched_rows.empty:
    print("Mismatched rows:")
    print(mismatched_rows[[aggregate_column, 'TEMP_SUM']])
else:
    TOINC_components = components

TOINC_components


In [ ]:
# List of possible component columns
components = [
    'BREAD', 'MEAT', 'FISH', 'MILK', 'OIL', 'FRUIT', 'VEG', 'SUGAR', 'FOOD_NEC',
    'FRUIT_VEG', 'COFFEE', 'TEA', 'COCOA', 'WATER', 'SOFTDRINKS', 'OTHER_NON_ALCOHOL'
]

loss_components = []
aggregate_column = 'FOOD_HOME'

# Call the function verify the aggregated column
mismatched_rows, result = check_aggregation(cleaned_fies, components, aggregate_column, negative_components=loss_components)

# Display the result
print(result)

# If there are mismatches, display them
if not mismatched_rows.empty:
    print("Mismatched rows:")
    print(mismatched_rows[[aggregate_column, 'TEMP_SUM']])
else:
    FOOD_HOME_components = components

FOOD_HOME_components

In [ ]:
# List of possible component columns
components = [
    'FOOD_HOME', 'FOOD_OUTSIDE'
]

aggregate_column = 'FOOD'
loss_components = []

# Call the function verify the aggregated column
mismatched_rows, result = check_aggregation(cleaned_fies, components, aggregate_column, negative_components=loss_components)

# Display the result
print(result)

# If there are mismatches, display them
if not mismatched_rows.empty:
    print("Mismatched rows:")
    print(mismatched_rows[[aggregate_column, 'TEMP_SUM']])
else:
    FOOD_components = components

FOOD_components

In [ ]:
# List of possible component columns
components = [
    'ALCOHOL', 'TOBACCO', 'OTHER_VEG', 'SERVICES_PRIMARY_GOODS', 
    'ALCOHOL_PROCDUCTION_SERVICES', 'CLOTH', 'HOUSING_WATER', 
    'FURNISHING', 'HEALTH','TRANSPORT',  'COMMUNICATION', 'RECREATION',
    'EDUCATION','INSURANCE','MISCELLANEOUS', 'DURABLE','OCCASION',
    'OTHER_EXPENDITURE', 'FOOD_ACCOM_SRVC',
]

aggregate_column = 'NFOOD'
loss_components = []

# Call the function verify the aggregated column
mismatched_rows, result = check_aggregation(cleaned_fies, components, aggregate_column, negative_components=loss_components)

# Display the result
print(result)

# If there are mismatches, display them
if not mismatched_rows.empty:
    print("Mismatched rows:")
    print(mismatched_rows[[aggregate_column, 'TEMP_SUM']])
else:
    NFOOD_components = components

NFOOD_components

In [ ]:
# List of possible component columns
components = [
    'FOOD', 'NFOOD'
]

aggregate_column = 'TOTEX'
loss_components = []

# Call the function verify the aggregated column
mismatched_rows, result = check_aggregation(cleaned_fies, components, aggregate_column, negative_components=loss_components)

# Display the result
print(result)

# If there are mismatches, display them
if not mismatched_rows.empty:
    print("Mismatched rows:")
    print(mismatched_rows[[aggregate_column, 'TEMP_SUM']])
else:
    TOTEX_components = components

TOTEX_components

In [ ]:
# List of possible component columns
components = [
    'TOTEX', 'OTHER_DISBURSEMENT'
]

aggregate_column = 'TOTDIS'
loss_components = []

# Call the function verify the aggregated column
mismatched_rows, result = check_aggregation(cleaned_fies, components, aggregate_column, negative_components=loss_components)

# Display the result
print(result)

# If there are mismatches, display them
if not mismatched_rows.empty:
    print("Mismatched rows:")
    print(mismatched_rows[[aggregate_column, 'TEMP_SUM']])
else:
    TOTDIS_components = components

TOTDIS_components

In [ ]:
# List of possible component columns
components = [
    'TOINC', 'OTHREC'
]

aggregate_column = 'TOREC'
loss_components = []

# Call the function verify the aggregated column
mismatched_rows, result = check_aggregation(cleaned_fies, components, aggregate_column, negative_components=loss_components)

# Display the result
print(result)

# If there are mismatches, display them
if not mismatched_rows.empty:
    print("Mismatched rows:")
    print(mismatched_rows[[aggregate_column, 'TEMP_SUM']])
else:
    TOREC_components = components

TOREC_components

In [ ]:
# Get all column names of the cleaned_fies DataFrame
cleaned_fies_columns = cleaned_fies.columns.tolist()

# Print the list of column names
print("Complete columns in cleaned_fies:", cleaned_fies_columns)


In [ ]:
# Printing the component lists for each aggregated column
print("WAGES components:", Wages_components)
print("EAINC components:", EAINC_components)
print("TOINC components:", TOINC_components)
print("FOOD_HOME components:", FOOD_HOME_components)
print("FOOD components:", FOOD_components)
print("NFOOD components:", NFOOD_components)
print("TOTEX components:", TOTEX_components)
print("TOTDIS components:", TOTDIS_components)
print("TOREC components:", TOREC_components)

In [ ]:
# Function to extract names and parents from hierarchical data
def extract_hierarchy(data, parent="Root"):
    names = []
    parents = []
    for key, value in data.items():
        names.append(key)
        parents.append(parent)
        if isinstance(value, dict):
            child_names, child_parents = extract_hierarchy(value, key)
            names.extend(child_names)
            parents.extend(child_parents)
    return names, parents

# Extract names and parents from the dataset hierarchy
names, parents = extract_hierarchy({
    "W_REGN": None,
    "W_PROV": None,
    "SEQ_NO": None,
    "RPROV": None,
    "FSIZE": None,
    "TOREC": {
        "TOINC": {
            "WAGES": {
                "REG_SAL": None,
                "SEASON_SAL": None
            },
            "NETSHARE": None,
            "CASH_ABROAD": None,
            "CASH_DOMESTIC": None,
            "RENTALS_REC": None,
            "INTEREST": None,
            "PENSION": None,
            "DIVIDENDS": None,
            "OTHER_SOURCE": None,
            "NET_RECEIPT": None,
            "REGFT": None,
            "IMPUTED_RENT": None,
            "EAINC": {
                "NET_CFG": None,
                "NET_LPR": None,
                "NET_FISH": None,
                "NET_FOR": None,
                "NET_RET": None,
                "NET_MFG": None,
                "NET_TRANS": None,
                "NET_NEC_A8": None,
                "NET_NEC_A9": None,
                "NET_NEC_A10": None,
                "LOSSES": None
            }
        },
        "OTHREC": None
    },
    "TOTDIS": {
        "TOTEX": {
            "FOOD": {
                "FOOD_HOME": {
                    "BREAD": None,
                    "MEAT": None,
                    "FISH": None,
                    "MILK": None,
                    "OIL": None,
                    "FRUIT": None,
                    "VEG": None,
                    "SUGAR": None,
                    "FOOD_NEC": None,
                    "FRUIT_VEG": None,
                    "COFFEE": None,
                    "TEA": None,
                    "COCOA": None,
                    "WATER": None,
                    "SOFTDRINKS": None,
                    "OTHER_NON_ALCOHOL": None
                },
                "FOOD_OUTSIDE": None
            },
            "NFOOD": {
                "ALCOHOL": None,
                "TOBACCO": None,
                "OTHER_VEG": None,
                "SERVICES_PRIMARY_GOODS": None,
                "ALCOHOL_PROCDUCTION_SERVICES": None,
                "CLOTH": None,
                "HOUSING_WATER": None,
                "FURNISHING": None,
                "HEALTH": None,
                "TRANSPORT": None,
                "COMMUNICATION": None,
                "RECREATION": None,
                "EDUCATION": None,
                "INSURANCE": None,
                "MISCELLANEOUS": None,
                "DURABLE": None,
                "OCCASION": None,
                "OTHER_EXPENDITURE": None,
                "FOOD_ACCOM_SRVC": None
            }
        },
        "OTHER_DISBURSEMENT": None
    },
    "RPSU": None,
    "RFACT": None,
    "MEM_RFACT": None,
    "URB": None,
    "PERCAPITA": None,
    "NPCINC": None,
    "RPCINC": None,
    "PRPCINC": None,
    "PPCINC": None,
    "RPCINC_NIR": None,
    "W_REGN_NIR": None
})



In [ ]:
# Create the treemap visualization
fig = px.treemap(
    names=names,
    parents=parents,
    title="Hierarchy of Columns in FIES Dataset",
    color_discrete_sequence=["#636EFA", "#EF553B", "#00CC96", "#AB63FA"]

)
fig.update_traces(root_color="lightgrey")
fig.update_layout(
    width=1400,  # Set the width of the figure
    height=800,  # Set the height of the figure
    margin=dict(t=50, l=25, r=25, b=25)  # Adjust the margins if needed
)
fig.show()

In [ ]:
# Get all column names of the cleaned_fies DataFrame
cleaned_fies_columns = cleaned_fies.columns.tolist()

# Print the list of column names
print("Complete columns in cleaned_fies:", cleaned_fies_columns)

# Printing the component lists for each aggregated column
print("WAGES components:", Wages_components)
print("EAINC components:", EAINC_components)
print("TOINC components:", TOINC_components)
print("FOOD_HOME components:", FOOD_HOME_components)
print("FOOD components:", FOOD_components)
print("NFOOD components:", NFOOD_components)
print("TOTEX components:", TOTEX_components)
print("TOTDIS components:", TOTDIS_components)
print("TOREC components:", TOREC_components)


In [ ]:
def plot_correlation_heatmap(dataframe, aggregated_column, components, title):
    # Subset the dataframe
    subset = dataframe[[aggregated_column] + components]
    
    # Calculate the correlation matrix
    correlation_matrix = subset.corr()
    
    # Plot heatmap
    plt.figure(figsize=(15, 10))
    sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm", cbar=True, square=True)
    plt.title(title)
    plt.tight_layout()
    plt.show()

In [ ]:
def plot_grouped_bar_chart_average(dataframe, components, group_by_column, title):
    # Calculate average values for components grouped by the specified column
    averages = dataframe.groupby(group_by_column)[components].mean().reset_index()
    
    # Melt the dataframe for easier plotting
    melted = averages.melt(id_vars=[group_by_column], 
                           value_vars=components, 
                           var_name='Component', 
                           value_name='Average Value')
    
    # Plot the grouped bar chart
    plt.figure(figsize=(12, 6))
    sns.barplot(data=melted, x=group_by_column, y='Average Value', hue='Component', palette='viridis')
    plt.title(title)
    plt.ylabel('Average Value')
    plt.xlabel(group_by_column)
    plt.legend(title='Component')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


In [ ]:
# Define the aggregated column and its components
aggregated_col = 'WAGES'
components_list = Wages_components
title = f'Average Values of Wages by region'

# Call the function
plot_grouped_bar_chart_average(cleaned_fies, components_list, group_by_column='W_REGN', title=title)


In [ ]:
# Define the aggregated column and its components
aggregated_col = 'EAINC'
components_list = EAINC_components
title = f'Heatmap Correlation: {aggregated_col} and Its Components'

# Call the function
plot_correlation_heatmap(cleaned_fies, aggregated_col, components_list, title=title)


In [ ]:
# Define the aggregated column and its components
aggregated_col = 'TOINC'
components_list = TOINC_components
title = f'Heatmap Correlation: {aggregated_col} and Its Components'

# Call the function
plot_correlation_heatmap(cleaned_fies, aggregated_col, components_list, title=title)


In [ ]:
# Define the aggregated column and its components
aggregated_col = 'FOOD_HOME'
components_list = FOOD_HOME_components
title = f'Heatmap Correlation: {aggregated_col} and Its Components'

# Call the function
plot_correlation_heatmap(cleaned_fies, aggregated_col, components_list, title=title)


In [ ]:
# Define the aggregated column and its components
aggregated_col = 'NFOOD'
components_list = NFOOD_components
title = f'Heatmap Correlation: {aggregated_col} and Its Components'

# Call the function
plot_correlation_heatmap(cleaned_fies, aggregated_col, components_list, title=title)


In [ ]:
# Define the aggregated column and its components
aggregated_col = 'FOOD'
components_list = FOOD_components
title = f'Average Values of Food Expenses by region'

# Call the function
plot_grouped_bar_chart_average(cleaned_fies, components_list, group_by_column='W_REGN', title=title)


In [ ]:
# Define the aggregated column and its components
aggregated_col = 'TOTEX'
components_list = TOTEX_components
title = f'Average Values of Total Expenses by region'

# Call the function
plot_grouped_bar_chart_average(cleaned_fies, components_list, group_by_column='W_REGN', title=title)
